<h1>Japanese Onomatopoeia Paraphraser</h1>

## Dataset

A. Corpus for Japanese including onomatopoeia
- https://huggingface.co/datasets/oscar-corpus/OSCAR-2201

B. Article for onomatopoeia list
- https://www.tufs.ac.jp/common/fs/ilr/contents/ronshuu/26/jilr26_Article_Huang.pdf

C. Onomatopoeia website
- https://www2.ninjal.ac.jp/Onomatope/50_on/gatagata.html


In [2]:
#download libraries
import numpy as np
import re
import random
from gensim.models import Word2Vec
import MeCab
from datasets import load_dataset


In [18]:


#OnomatoParaphraser

class OnomatoParaphraser:
    #initialize 
    #The tools I would need are "tokens and POS" to look into the original sentence 
    #                           "onomatope list" that I found in an article
    #                           "Word2Vec model" 
    def __init__(self, onomatope_path, additional_path):
        self.mecab = MeCab.Tagger("--rcfile=/opt/homebrew/etc/mecabrc") 
        self.word2vec_model = None

        with open(onomatope_path, 'r', encoding='utf-8') as f:
            onomatope_lines = f.readlines()
            
        self.onomatope_list = []
        for line in onomatope_lines:
            onomatopes = line.strip().split()
            self.onomatope_list.extend(onomatopes)

        self.onomatope_list = list(set(self.onomatope_list))
        print(f"Loaded {len(self.onomatope_list)} onomatopoeia from the list")
                
        with open(additional_path, 'r', encoding='utf-8') as f:
            additional_onoma_sentences = f.readlines()

        self.additional_onoma_sentences = [line.strip().strip(',') for line in additional_onoma_sentences if line.strip()]
        print(f"Loaded {len(self.additional_onoma_sentences)} additional onomatopoeia sentences")

    
    #Mecab output format
    #表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音


    
    #Function to tokenize Japanese text with MeCab
    def tokenize_text(self, text):
        parsed_text = self.mecab.parse(text)
        lines = parsed_text.split('\n')
        tokens = []

        for line in lines:
            if line == 'EOS' or line == '':
                continue
            parts = line.split('\t')
            if len(parts) >= 2:
                tokens.append(parts[0])

        return tokens

    

    #Analyze POS because depending on POS, onomatope gets different suffix
    def analyze_sentence(self, sentence):
        mecab_result = self.mecab.parse(sentence)
        lines = mecab_result.split('\n')

        words = []
        POS_tags = []
        
        for line in lines:
            if line == 'EOS' or line == '':
                continue
            parts = line.split('\t')
            if len(parts) >= 2:
                word = parts[0]
                pos = parts[1].split(',')[0] #mecab.parse includes several information, and the first one is POS

                words.append(word)
                POS_tags.append(pos)
        

        return list(zip(words, POS_tags))
        

 #load Japanese texts from the corpus
    def load_dataset(self, max_sample):

        print(f"loading corpus dataset, max={max_sample}")

        dataset = load_dataset("oscar-corpus/OSCAR-2301",
                               language="ja",
                               streaming=True,
                               split="train",
                              )
        
        
        #RegEx to remove some symbols
        symbol_pattern = re.compile(r'[■□※●○\{\}【】\*]+')
        
        #save Japanese sentences
        JP_sentences = []
        count = 0

        for sample in dataset:

            #The corpus is Dict["id":___, "text":___]
            text = sample["text"] 

            #I want to split with \n and '。'
            lines = text.split('\n')
            
            for line in lines:
                sentences = line.split('。')

            #if the sentence has a symbol, I will not use it. Go to next sentence
            for sentence in sentences:
                if symbol_pattern.search(sentence):
                    continue
                
            #if the sentence is too short or too long, should be removed
                if 10 <= len(sentence) <= 140 and sentence.strip():
                    JP_sentences.append(sentence)
                    count += 1

                    if count >= max_sample:
                        print(f"Loaded {len(JP_sentences)} sentences")
                        return JP_sentences

 
        print(f"Loaded {len(JP_sentences)} sentences from corpus")
        return JP_sentences

        
                    


    
    #tokenize sentences as preparation for the word2vec model
    def prepare_data_word2vec(self, training_sentences, max_sentences):
        print("preparing data for word2vec")

        #I want to make tokenized_sentences list and count the number of sentence
        tokenized_sentences = []
        count = 0

        #if the number of sentence is over, should be break
        for sentence in training_sentences:
            if count >= max_sentences:
                break

            #tokenize text and add to tokenized_sentences list
            tokens = self.tokenize_text(sentence)
            tokenized_sentences.append(tokens)
            count += 1

        print(f"Completed preparation. The number of sentence for word2vec: {count}")
        return tokenized_sentences
        
    
    #train word2vec
    def train_word2vec(self, tokenized_sentences):
        print("Training Word2Vec")

        self.word2vec_model = Word2Vec(
            sentences=tokenized_sentences,
            vector_size=100,
            window=5,
            min_count=1,
            sg=1, #skip_gram
            epochs=10
        )

        print(f"completed training word2vec model with {len(tokenized_sentences)} sentences")
       
        #if the onomatope in my list is not in the word2vec model, 
        #adding random vector to the model for the onomatope
        
        onoma_in_vocab = 0
        onoma_added = 0

        new_vectors = []
        new_words = []
        
        for onomatope in self.onomatope_list:
            if onomatope in self.word2vec_model.wv:
                onoma_in_vocab +=1
            else:
                random_vector = np.random.uniform(-0.25, 0.25, size=self.word2vec_model.vector_size)
                new_words.append(onomatope)
                new_vectors.append(random_vector)
                onoma_added += 1

        if new_words:
            self.word2vec_model.wv.add_vectors(new_words, new_vectors)
        
        print(f"onomatope in word2vec vocab: {onoma_in_vocab}, onomatope random vector added:{onoma_added}")

        return self.word2vec_model




    #using the vectors from word2Vec, find similar onomatopoeias for the word
    def find_similar_onomatope(self, word, top_n=5):

        if word not in self.word2vec_model.wv:
            print(f"'{word}' is not in the vocabulary.")
            return []
        
        #extract similarity of word from sentence and onomatope from onomatope list in word2vec
        similarities = []
        for onoma in self.onomatope_list:
            if onoma in self.word2vec_model.wv:
                sim = self.word2vec_model.wv.similarity(word, onoma)
                similarities.append((onoma, sim))

                
        #sort from high score
        similarities.sort(key=lambda x: x[1], reverse=True)

        return similarities[:top_n]


    #paraphrase the sentence
    def paraphrase_sentence(self, sentence):
        #analyze the sentence
        analyzed_sentence = self.analyze_sentence(sentence)

        #paraphrased result [original sentence;__, new sentence:__, similarity:__]
        paraphrased_result = []

        #Conditions
        #1. If the sentence contains all '副詞', '形容詞', and '動詞' in this order, paraphrase the  '形容詞'.
        #2. If the sentence contains '副詞' and '動詞' in this order, paraphrase the  '動詞'.
        #3. If the sentence contains '形容詞' without '動詞', paraphrase the '形容詞'.

        #The method has not yet found the POS
        adv_pos = -1
        adj_pos = -1
        v_pos = -1

        #find the index of the POS
        for i, (word, pos) in enumerate(analyzed_sentence):
            if pos == '副詞':
                if adv_pos == -1:
                    adv_pos = i
            elif pos == '形容詞':
                if adj_pos == -1:
                    adj_pos = i
            elif pos == '動詞':
                if v_pos == -1:
                    v_pos = i

        #1. If the sentence contains all '副詞', '形容詞', and '動詞' in this order, paraphrase the  '形容詞'.
        if (
            adv_pos != -1 and
            adj_pos != -1 and
            v_pos != -1 and
            adv_pos +1 == adj_pos and
            adj_pos +1 == v_pos
        ):
            target_pos = adj_pos
            target_word, pos = analyzed_sentence[target_pos]

        #2. If the sentence contains '副詞' and '動詞' in this order, paraphrase the  '動詞'.
        elif adv_pos != -1 and v_pos != -1 and adv_pos +1 == v_pos:
            target_pos = v_pos
            target_word, pos = analyzed_sentence[target_pos]

        #3. If the sentence contains '形容詞' without '動詞', paraphrase the '形容詞'.
        elif adj_pos != -1 and v_pos == -1:
            target_pos = adj_pos
            target_word, pos = analyzed_sentence[target_pos]

        #4. If only verb here 
        elif v_pos != -1 and adj_pos == -1 and adv_pos == -1:
            target_pos = v_pos
            target_word, pos = analyzed_sentence[target_pos]
            mode = "insert"

        #Else, return []
        else:
            return paraphrased_result
            
    
        #find similar onomatopoeia    
        similar_onomatope = self.find_similar_onomatope(target_word, top_n=5)
   

        #choose most similar one
        if not similar_onomatope:
            return paraphrased_result
        
        onoma, sim = similar_onomatope[0]

        #if the similarity is too low, I would skip
        if sim < 0.05:
            return paraphrased_result

        #find similar onomatope for the target(ind, word, pos)
        #switch it to the onomatope
        words = [w for w, _ in analyzed_sentence]

        if mode == "insert":
            words.insert(target_pos, onoma)
        else:
            words[target_pos] = onoma

        new_sentence = "".join(words)
        
        #return the paraphrased sentence        
        paraphrased_result.append({
            'original sentence': sentence,
            'paraphrased sentence': new_sentence,
            'similarity': sim
        })


        return paraphrased_result
    


    #'prepare_and_train' method prepare for this whole model and train
    #>>> This part does: set onomatope list, prepare training data, train word2vec
    def prepare_and_train(self, training_sentences):
        
        random.seed(42)
   

        #Word2Vec training data
        training_data = self.prepare_data_word2vec(training_sentences, max_sentences=500000)
        
        #train word2vec
        self.train_word2vec(training_data)
        
        print("completed training model")

        
        

    


__-How to implement the python code__


    

In [19]:
def main():
    #https://www.tufs.ac.jp/common/fs/ilr/contents/ronshuu/26/jilr26_Article_Huang.pdf

    #path to onomatope_list
    onomatope_path = "/Users/daikisuematsu/LING539/onomatopoeia-nlp/datasets/onomatope_list.txt"
    
    #path to onomatope_sentences
    additional_path = "/Users/daikisuematsu/LING539/onomatopoeia-nlp/datasets/onomatope_sentences.txt"

    #prepare paraphraser
    paraphraser = OnomatoParaphraser(onomatope_path, additional_path)

    
    #prepare corpus data for training
    print("loading corpus data")
    training_sentences = paraphraser.load_dataset(max_sample=100000)

    #prepare more training sentences of onomatopoeia for training
    training_sentences.extend(paraphraser.additional_onoma_sentences)
    
    #prepare_and_train(self, onomatope_list, training_sentences):
    paraphraser.prepare_and_train(training_sentences)

    #input test_sentence
    test_sentence = input("please input a sentence that you want to paraphrase:")
    results = paraphraser.paraphrase_sentence(test_sentence)
       # paraphrased_result.append({
        #            'original sentence': sentence,
         #           'paraphrased sentence': new_sentence,
          #          'similarity': sim
           #     })
    
    if results:
        for result in results:
            print("original sentence:", result['original sentence'])
            print("paraphrased sentence:", result['paraphrased sentence'])
            print("similarity:", result['similarity'])
    else:
        print("Failed to paraphrase")

if __name__ == "__main__":
    main()
    

Loaded 302 onomatopoeia from the list
Loaded 89 additional onomatopoeia sentences
loading corpus data
loading corpus dataset, max=100000
Loaded 100000 sentences
preparing data for word2vec
Completed preparation. The number of sentence for word2vec: 100089
Training Word2Vec
completed training word2vec model with 100089 sentences
onomatope in word2vec vocab: 172, onomatope random vector added:130
completed training model


please input a sentence that you want to paraphrase: 彼は歩く


original sentence: 彼は歩く
paraphrased sentence: 彼はこつこつ歩く
similarity: 0.66275483


In [20]:
#Prepare for testing the word2vec model
onomatope_path = "/Users/daikisuematsu/LING539/onomatopoeia-nlp/datasets/onomatope_list.txt"
additional_path = "/Users/daikisuematsu/LING539/onomatopoeia-nlp/datasets/onomatope_sentences.txt"

paraphraser = OnomatoParaphraser(onomatope_path, additional_path)

training_sentences = paraphraser.load_dataset(max_sample=100000)
training_sentences.extend(paraphraser.additional_onoma_sentences)

paraphraser.prepare_and_train(training_sentences)

Loaded 302 onomatopoeia from the list
Loaded 89 additional onomatopoeia sentences
loading corpus dataset, max=100000
Loaded 100000 sentences
preparing data for word2vec
Completed preparation. The number of sentence for word2vec: 100089
Training Word2Vec
completed training word2vec model with 100089 sentences
onomatope in word2vec vocab: 172, onomatope random vector added:130
completed training model


In [21]:
#test the word2vec model
def test_similarity(paraphraser, top_n=5):

    test_word = input("please input a word").strip()

    print(f"Similar onomatopoeia of {test_word}  Top {top_n}：")
    similar = paraphraser.find_similar_onomatope(test_word, top_n=5)

    if not similar:
        print("no words in word2vec")
        return
    
    for onoma, sim in similar:
        print(f"{onoma}: {sim:.4f}")
        

            
            

In [55]:
#test the model
test_similarity(paraphraser)

please input a word 食べる


Similar onomatopoeia of 食べる  Top 5：
からから: 0.7096
ワイワイ: 0.7022
さっと: 0.6892
がっかり: 0.6867
さっぱり: 0.6839


# Result

The model generated the following outputs. When the target word was either an adverb or an adjective, the model replaced it with an onomatopoeia. For verbs, the model inserted an onomatopoeia immediately before the verb.
As a native Japanese speaker, I evaluated each suggested onomatopoeia for semantic appropriateness. In patterns 1 and 2 (involving adverbs and adjectives), I marked with an "S" those onomatopoeia that were somewhat semantically similar to the target word's meaning within a context. For pattern 3 (involving verbs), I marked with an "S" those cases where the onomatopoeia appropriately matched the verb. Cases that were semantically inappropriate were marked with an "X".

The accuracy rates for each pattern were:

- **Result1(Adverbs)**:8/30(26%)
- **Result2(Adjectives)**: 7/30(23%)
- **Result3(Verbs)**: 5/35(14%)




## Pattern 1: Adverb to onomatopoeia
Similar onomatopoeia of 遅く(slowly, lately)  Top 5：
- ジメジメ: 0.6864 | x
- ぐったり: 0.6815 | x
- ぐっすり: 0.6734 | x
- ぎりぎり: 0.6651 | S
- メチャメチャ: 0.6647 | x

Similar onomatopoeia of おそく(slowly, lately)  Top 5：
- メチャメチャ: 0.8546 | x
- ガックリ: 0.8528 | x
- カンカン: 0.8342 | x
- ぐしゃぐしゃ: 0.8319 | x
- ごつごつ: 0.8303 | x

Similar onomatopoeia of 早く(quickly)  Top 5：
- イライラ: 0.6549 | x
- ぐっすり: 0.6529 | x
- がっかり: 0.6280 | x
- ジメジメ: 0.6212 | x
- モリモリ: 0.6148 | x

Similar onomatopoeia of はやく(quickly)  Top 5：
- コッテリ: 0.8709 | x
- ザラザラ: 0.8703  | x
- ギクシャク: 0.8602 | x
- アッサリ: 0.8563 | S
- ウロウロ: 0.8549 | x

Similar onomatopoeia of 激しく(aggresively, intensely)  Top 5：
- ぐんぐん: 0.7755 | x
- グイグイ: 0.7667 | S
- ジメジメ: 0.7504 | x
- ぐっと: 0.7358 | S
- イライラ: 0.7255 | x

Similar onomatopoeia of はげしく(aggresively, intensely)  Top 5：
'はげしく' is not in the vocabulary.
no words in word2vec

Similar onomatopoeia of 強く(strongly)  Top 5：
- ぐんぐん: 0.7068 | S
- モリモリ: 0.6954 | S
- ぐっと: 0.6867 | x
- くっきり: 0.6619 | S
- グイグイ: 0.6534 | S

Similar onomatopoeia of つよく(strongly)  Top 5：
'つよく' is not in the vocabulary.
no words in word2vec

**Result1**:8/30(26%)

## Pattern 2: Adjective to onomatopoeia
Similar onomatopoeia of きれい(beautiful)  Top 5：
- モリモリ: 0.6682 | x
- クルクル: 0.6584 | x
- くっきり: 0.6508 | S
- キョロキョロ: 0.6408 | x
- さっと: 0.6403 | x

Similar onomatopoeia of やわらかい(soft)  Top 5：
- しっとり: 0.8759 | x
- こんがり: 0.8232 | x
- ゴシゴシ: 0.8223 | x
- ほんのり: 0.8070 | x
- シャキシャキ: 0.8053 | x

Similar onomatopoeia of 静か(quiet)  Top 5：
- モリモリ: 0.6682 | x
- ぐったり: 0.6607 | x
- わいわい: 0.6499 | x
- ぐいぐい: 0.6348 | x
- おっとり: 0.6257 | S

Similar onomatopoeia of しずか(quiet)  Top 5：
'しずか' is not in the vocabulary.
no words in word2vec

Similar onomatopoeia of 楽しい(fun)  Top 5：
- ワクワク: 0.7128 | S
- ワイワイ: 0.6901 | S
- めちゃめちゃ: 0.6825 | x
- ぐっと: 0.6513 | x
- ころころ: 0.6489 | x

Similar onomatopoeia of たのしい(fun)  Top 5：
- うだうだ: 0.7752 | x
- わいわい: 0.7744 | S
- シッカリ: 0.7653 | x
- ポンポン: 0.7573 | x
- もりもり: 0.7539 | x

Similar onomatopoeia of にぎやか(lively)  Top 5：
- げっそり: 0.8616 | x
- ぐんぐん: 0.8461 | x
- わいわい: 0.8436 | S
- メチャメチャ: 0.8412 | x
- ワイワイ: 0.8392 | S

**Result2**: 7/30(23%)

## Pattern 3: Add an onomatopoeia in front of the verb(dictionary form) 

Similar onomatopoeia of 歩く(to walk)  Top 5：
- じっと: 0.6458 | x
- こつこつ: 0.6444 | S
- きらきら: 0.6287 | x
- うろうろ: 0.6197 | S
- げっそり: 0.6164 | x

Similar onomatopoeia of あるく(to walk)  Top 5：
- ザラザラ: 0.9031 | x
- ガサガサ: 0.8997 | x
- うんと: 0.8996 | x
- ゴツゴツ: 0.8946 | x
- ゴタゴタ: 0.8943 | x

Similar onomatopoeia of 降る(to rain)  Top 5：
- ジメジメ: 0.7663 | x
- うんざり: 0.7361 | x
- ぐったり: 0.7345 | x
- じめじめ: 0.7303 | x
- ころころ: 0.7191 | x

Similar onomatopoeia of 揺れる(to quake)  Top 5：
- ごろごろ: 0.8578 | x
- ころころ: 0.8460 | x
- くるくる: 0.8312 | x
- くっきり: 0.8299 | x
- ほんのり: 0.8298 | x

Similar onomatopoeia of ゆれる(to quake)  Top 5：
- げんなり: 0.8542 | x
- グッタリ: 0.8527 | x
- コッテリ: 0.8462 | x
- がさがさ: 0.8450 | S
- ぎくしゃく: 0.8398 | x

Similar onomatopoeia of 笑う(to laugh)  Top 5：
- げらげら: 0.8411 | S
- ガンガン: 0.8390 | x
- ワイワイ: 0.8382 | x
- がっかり: 0.8365 | x
- めちゃめちゃ: 0.8344 | S

Similar onomatopoeia of 言う(to say)  Top 5：
- ぼんやり: 0.6887 | x
- じめじめ: 0.6785 | x
- めちゃくちゃ: 0.6706 | x
- モリモリ: 0.6680 | x
- メチャクチャ: 0.6660 | x


**Result3**: 5/35(14%)


## Pattern 3.5: Add an onomatopoeia in front of the verb(present progressive form) 

Similar onomatopoeia of 歩いています(is walking)  Top 5：
'歩いています' is not in the vocabulary.
no words in word2vec

Similar onomatopoeia of あるいています(is walking)  Top 5：
'あるいています' is not in the vocabulary.
no words in word2vec

Similar onomatopoeia of 揺れている(is quaking)  Top 5：
'揺れている' is not in the vocabulary.
no words in word2vec

Similar onomatopoeia of ゆれている(is quaking)  Top 5：
'ゆれている' is not in the vocabulary.
no words in word2vec





In [ ]:
"""
import jaconv

katakana_text = """アタフタ アッサリ アヤフヤ イソイソ イライラ ウキウキ ウジャウジャ 
        ウダウダ ウッカリ ウッスラ ウットリ ウツラウツラ ウトウト ウロウロ ウンザリ ウント オイオイ オズオズ
        オソルオソル オットリ カサカサ ガサガサ カタカタ ガタガタ カチカチ ガチャガチャ ガツガツ 
        ガッカリ ガックリ ガッチリ カット ガミガミ カラカラ カラット ガラリト カリカリ
        ガリガリ カンカン ガンガン ギクシャク ギザギザ ギスギス キチント ギッシリ キッチリ キット キッパリ キビキビ
        ギュット ギョット キョロキョロ キラキラ ギラギラ ギリギリ グイグイ グウグウ クシャクシャ グシャグシャ クスクス クタクタ
        グチャグチャ クッキリ グツグツ グッスリ グッタリ グット クヨクヨ グラグラ クリクリ クルクル グルグル クルリ
        グングン グント ゲッソリ ゲラゲラ ケロット ゲンナリ ゴクゴク ゴシゴシ コソコソ ゴソゴソ ゴタゴタ ゴチャゴチャ
        コツコツ ゴツゴツ コッソリ ゴッチャ コッテリ コトコト コロコロ ゴロゴロ ゴワゴワ コンガリ コンコン コンモリ
        サクサク ザックバラン ザックリ サッサト サット ザット サッパリ サラサラ ザラザラ ザワザワ シクシク シゲシゲ
        シッカリ シックリ ジックリ ジット シットリ シトシト シバシバ ジメジメ シャキシャキ シャックリ シャブシャブ ジャラジャラ
        アタフタ アッサリ アヤフヤ イソイソ イライラ ウキウキ ウジャウジャ ウダウダ ウッカリ ウッスラ ウットリ ウツラウツラ
        ウトウト ウロウロ ウンザリ ウント オイオイ オズオズ オソルオソル オットリ カサカサ ガサガサ カタカタ ガタガタ
        カチカチ ガチャガチャ ガツガツ ガッカリ ガックリ ガッチリ カット ガミガミ カラカラ カラット ガラリト カリカリ
        ガリガリ カンカン ガンガン ギクシャク ギザギザ ギスギス キチント ギッシリ キッチリ キット キッパリ キビキビ
        ギュット ギョット キョロキョロ キラキラ ギラギラ ギリギリ グイグイ グウグウ クシャクシャ グシャグシャ クスクス クタクタ
        グチャグチャ クッキリ グツグツ グッスリ グッタリ グット クヨクヨ グラグラ クリクリ クルクル グルグル クルリ
        グングン グント ゲッソリ ゲラゲラ ケロット ゲンナリ ゴクゴク ゴシゴシ コソコソ ゴソゴソ ゴタゴタ ゴチャゴチャ
        コツコツ ゴツゴツ コッソリ ゴッチャ コッテリ コトコト コロコロ ゴロゴロ ゴワゴワ コンガリ コンコン コンモリ
        サクサク ザックバラン ザックリ サッサト サット ザット サッパリ サラサラ ザラザラ ザワザワ シクシク シゲシゲ
        シッカリ シックリ ジックリ ジット シットリ シトシト シバシバ ジメジメ シャキシャキ シャックリ シャブシャブ ジャラジャラ
        ホトホト ホノボノ ボヤボヤ ボロボロ ポロポロ ホンノリ ポンポン ボンヤリ マチマチ ムカムカ ムシャクシャ ムッツリ メチャ メチャクチャ
        メチャメチャ モクモク モタモタ モリモリ モロモロ ヤキモキ ヤンワリ ユックリ ユッタリ ヨタヨタ ヨチヨチ ヨボヨボ
        ヨレヨレ ヨロヨロ ワイワイ ワクワク ワンワン"""

# 単語ごとに分割して変換
words = katakana_text.split()
hiragana_words = [jaconv.kata2hira(word) for word in words]

# 結果表示
hiragana_text = " ".join(hiragana_words)
print(hiragana_text)
"""

In [57]:
'''
text = """
不合格の知らせにがっかりとさせられた。仕事がうまくいかなくてがっかりだ。生徒たちががやがやさわいでいる。
パーティ会場はがやがやしていた。がやがやした店は好きじゃない。のどがかわいて、からからだ。
洗濯物がからからに乾いた。からから天気の日が続く。がらがらとシャッターを開ける。
地震でへいががらがらと崩れた。歌いすぎてのどががらがらになった。家に帰ったらすぐ，がらがらとうがいをする。
店員ががらがら声で客を呼びこんでいる。がんがん工事をする音が聞こえる。野球のコーチががんがん怒鳴っている。
二日酔いで頭ががんがんする。クーラーをがんがんにきかせる。毎日朝ごはんをきちんと食べている。本棚に本をきちんと並べる。
家賃を毎月きちんと払う。財布にお札がぎっしりつまっている。今週は予定がぎっしりだ。
小さな字でぎっしりと書いてある。家具のサイズをきっちり測る。本棚に本がきっちり入れてある。
借りたお金はきっちり返す。セールスの電話はきっぱり断る。今日からたばこはきっぱりやめる。
きっぱりとした態度をとる。空の星がきらきら光る。子供は目がきらきらしている。
パーティにきらきらの服を着ていく。走っていったらぎりぎり間に合った。
合格点ぎりぎりでパスした。ぎりぎりのお金しか持っていかない。
締め切りぎりぎりにならないと書き始めない。マンガを読んでくすくすと笑っている。
くすくす笑いをしている。ぐずぐずしていると間に合わない。花粉症で鼻がぐずぐずする。きのうの夜は、ぐっすり眠れた。
子供がぐっすりと寝ている。最近どうもぐっすり寝られない。重いドアをぐっと押す。
泣きたくてもぐっとがまんする。冷たいビールをぐっと飲む。コマがくるくる回る。カレンダーをくるくる丸めた。
くるくるに巻いた髪。道に迷ってぐるぐる歩き回った。
乗り物がぐるぐる回転する。酔っ払って目がぐるぐる回る。
ロープでぐるぐる巻きにする。病気でげっそりとやせた。
げっそりした表情で帰ってきた。
毎日同じものを食べてげっそりする。漫画を読んでげらげら笑っている。
げらげら笑いすぎてお腹が痛くなった。
人の失敗をげらげらと笑うのは失礼だ。引き出しの中がごちゃごちゃだ。
小さな店がごちゃごちゃと建っている。
ごちゃごちゃした模様は好きじゃない。
映画のストーリーがごちゃごちゃしてわかりにくい。テーブルの上の料理をこっそり食べた。
夜遅くこっそりと家を出た。ピンポン玉がころころところがる。ころころと太った子犬。
言うことがころころ変わる。
試合にころころ負ける。雷がごろごろなる。
猫がごろごろとのどを鳴らす。
お腹がごろごろする。
荷物がごろごろ転がる。雨がざあざあ降っている。
外はざあざあ降りの大雨だ。一人でさっさと帰ってしまった。
宿題をさっさと片付ける。５時になったらさっと帰る。
テーブルの上をさっと片付ける。
雨がさっと降ってすぐやんだ。資料にざっと目を通す。
バケツの水をざっとかける。
袋に入った米がざっとこぼれた。
ざっと計算して100万円はかかる。長い髪をさっぱりと短くした。
デザートはさっぱり(と)した果物が食べたい。
難しくてさっぱりわからない。髪がさらさらできれいだ。
さらさらの粉雪が降りつもっている。
さらさらっとサインする。
砂がさらさら(と)こぼれる。砂がざらざらと落ちた。
風が強い日は，床がざらざらになる。
おばあちゃんの手はざらざらだ。家の土台がしっかりしている。
子どもが母親にしっかりつかまっている。
若いときからしっかり貯金している。
しっかりした計画を立てる。将来のことをじっくり考えて決める。
じっくり煮込んだシチューを作る。痛くてもじっとがまんする。
心配でじっとしていられない。人をじろじろ見るのは失礼だ。
店員にじろじろ見られた。薬を飲んですっかり良くなった。
宿題のことをすっかり忘れていた
もうすっかり一人前の大人だ。部屋を片付けてすっきりした。
今朝は、すっきり起きられた。
すっきりしたデザインの洋服。冷たいものを飲むとすっとする。
音もなくすっと部屋に入ってきた。
悪者がつかまって，胸がすっとした。このペンはすらすら書ける。
難しい問題をすらすら解いた。駐車場に観光バスがずらりと並んでいる。
世界各国のワインをずらりとそろえている。重い荷物をずるずるとひきずる。
ずるずると返事をのばしている。
悪い仲間とずるずる付き合う。親子で声がそっくりだ。
これとそっくりのかばんを持っている。
本物とそっくりに作ってある。
どろぼうが金庫の中身をそっくり盗んだ。ワイングラスをそっと持つ。
うしろの出口からそっと帰る。
この問題には触れずにそっとしておこう。もう遅いからそろそろ帰ろう。
息子もそろそろ結婚を考える歳になった。
足が痛いのでそろそろと歩いている。観光客がぞろぞろ降りてきた。なべにお湯をたっぷり入れる。
時間がたっぷりある。
たっぷりした服を着る。だぶだぶのズボンが流行する。
お腹の肉がだぶだぶしている。食事の前にちゃんと手を洗う。
小さな子供たちがちゃんと並んでいる。床がつるつるしていて危ない。
つるつるの肌になる。スピーチをするとき、どきどきした。
走ったあとは、心臓がどきどきする。
テストの点を見るときはいつもどきどきだ。1日中歩き回ってどっと疲れた。
電車から人がどっと降りてくる。ドアをどんどんとたたく音がする。
祭りの太鼓をどんどんたたく。
日本語がどんどん上手になる。
駅前にマンションがどんどん建つ。
ジャングルの中をどんどん進む。赤ちゃんがにこにこ笑っている。
ちえ子さんはいつもにこにこしている。
ボーナスをもらってにこにこ顔だ。携帯メールを見ながらにやにやしている。
授業中にやにや笑っていて先生に怒られた。
急ににやにやした顔になった。プールの底がぬるぬるしている。
油がついて，手がぬるぬるすべる。
ぬるぬるした海草が気持ち悪い。納豆のねばねばが口につく。
ねばねばした食べ物は体にいい。行列がのろのろと進んだ。
のろのろしていると遅れるよ。
連休で高速道路はどこものろのろ運転だ。温泉に入ってのんびりする。
休みの日は、のんびりと過ごしたい。赤ちゃんが手足をばたばたさせる。
暑さで人がばたばた倒れる。
朝はみんなばたばたと出かけていく。
忙しくてばたばたしている。寝不足で頭がはっきりしない。
はっきりしない天気が続く。20年前の友人とばったり会った。
選手はゴールに入ったとたんばったり（と）倒れた。悪い夢を見てはっと目がさめた。
急ブレーキの音がしてはっとした。
車内アナウンスにはっとして飛び降りた。ぱっと見て決めた。
うわさがぱっと広まる。
ボーナスをぱっと使ってしまった。
売り上げがぱっとしない。木の葉がはらはらと散る。
涙がはらはらと落ちた。人がばらばらと飛び出してきた。
家族の食事の時間がばらばらだ。
ジグソーパズルをばらばらにする。
ばらばら死体が発見された。くつをぴかぴかにみがく。
ダイアモンドの指輪がぴかぴか(と)光る。急に大きな音がしてびっくりした。
あの人がどろぼうだったとは、びっくりだ。
おじいさんから、びっくり箱をもらった。くつのサイズがぴったり合った。
子供が母親にぴったりくっついている。
体にぴったりした服を着る。パンがふっくらと焼けた。モデルにしてはふっくらした体型だ。ふと後ろを見ると，先生が立っていた。
ふとしたことから知り合った。
忘れていた用事をふと思い出した。寝不足でふらふらする。
酔っ払いがふらふら歩いている。
ビデオで撮った画面がふらふら揺れる。
悪い友達にふらふらとついていく。
ふらふらと遊び歩いている。電気のひもがぶらぶら揺れる。
小さな子供が足をぶらぶらさせている。
街をぶらぶらと歩く。こわくて足がぶるぶるする。犬がぶるぶるっと体をふるわせた。お腹がすいてぺこぺこだ。
電話をかけながら、ぺこぺこ頭を下げる。
社長の前ではぺこぺこする。
アルミの皿がぺこぺこする。外国語がぺらぺらだ。
ぺらぺらよくしゃべる。ページをぺらぺらとめくる。
ぺらぺらの紙。頭がぼうっとしてきた。
ぼうっとテレビを見ていた。試験に合格してほっとした。
手術が無事に終わってほっと安心した。
頂上に着いてほっと一息つく。ぼろぼろの辞書を使っている。
試合でぼろぼろに負けた。車がぼろぼろだ。彼はぼんやりした人だ。
遠くの山がぼんやりと見える。
昔のことをぼんやりと覚えている。船酔いで胸がむかむかする。
食べ過ぎて胃がむかむかする。
弟に負けてむかむかした。Ｂ君の作文はめちゃくちゃだ。
地震で家がめちゃくちゃにこわれた。
あの店はめちゃくちゃな値段をつけている。
Ｔ大学に合格してめちゃくちゃうれしい。男の子たちがもりもり食べている。
やる気がもりもりわいてきた。
ボディビルの選手は筋肉もりもりだ。
もっとゆっくり話してください。
川の水がゆっくり流れている。
みんなゆっくりしたペースで走っている。おじいさんがよろよろ歩いている。
あっちへよろよろ，こっちへよろよろする。
ボクサーがよろよろと倒れる。初めて海外旅行に行くのでわくわくする。
入学の日をわくわくしながら待っている。
『ハリー・ポッター』はわくわくどきどきの映画だ。
"""

lines = text.strip().split("。")
quoted_lines = ['"' + line.strip() + '",' for line in lines]

for i in range(0, len(quoted_lines), 3):
    row = quoted_lines[i: i+3]
    print(" ". join(row))

'''

"不合格の知らせにがっかりとさせられた", "仕事がうまくいかなくてがっかりだ", "生徒たちががやがやさわいでいる",
"パーティ会場はがやがやしていた", "がやがやした店は好きじゃない", "のどがかわいて、からからだ",
"洗濯物がからからに乾いた", "からから天気の日が続く", "がらがらとシャッターを開ける",
"地震でへいががらがらと崩れた", "歌いすぎてのどががらがらになった", "家に帰ったらすぐ，がらがらとうがいをする",
"店員ががらがら声で客を呼びこんでいる", "がんがん工事をする音が聞こえる", "野球のコーチががんがん怒鳴っている",
"二日酔いで頭ががんがんする", "クーラーをがんがんにきかせる", "毎日朝ごはんをきちんと食べている",
"本棚に本をきちんと並べる", "家賃を毎月きちんと払う", "財布にお札がぎっしりつまっている",
"今週は予定がぎっしりだ", "小さな字でぎっしりと書いてある", "家具のサイズをきっちり測る",
"本棚に本がきっちり入れてある", "借りたお金はきっちり返す", "セールスの電話はきっぱり断る",
"今日からたばこはきっぱりやめる", "きっぱりとした態度をとる", "空の星がきらきら光る",
"子供は目がきらきらしている", "パーティにきらきらの服を着ていく", "走っていったらぎりぎり間に合った",
"合格点ぎりぎりでパスした", "ぎりぎりのお金しか持っていかない", "締め切りぎりぎりにならないと書き始めない",
"マンガを読んでくすくすと笑っている", "くすくす笑いをしている", "ぐずぐずしていると間に合わない",
"花粉症で鼻がぐずぐずする", "きのうの夜は、ぐっすり眠れた", "子供がぐっすりと寝ている",
"最近どうもぐっすり寝られない", "重いドアをぐっと押す", "泣きたくてもぐっとがまんする",
"冷たいビールをぐっと飲む", "コマがくるくる回る", "カレンダーをくるくる丸めた",
"くるくるに巻いた髪", "道に迷ってぐるぐる歩き回った", "乗り物がぐるぐる回転する",
"酔っ払って目がぐるぐる回る", "ロープでぐるぐる巻きにする", "病気でげっそりとやせた",
"げっそりした表情で帰ってきた", "毎日同じものを食べてげっそりする", "漫

In [ ]:

[
        "日本料理は味があっさりしている。", "こってりした味。", "あっさりしたデザインが好きだ。", "すっきりしたデザインの服。", "兄はあっさりした性格だ。", "チャンピオンがあっさりと負けた。",
        "長い時間待たされていらいらする。", "隣の部屋がうるさくていらいらする。", "ドライバーのいらいらがつのる。",
        "試験の点が悪いのは、うっかりミスが多いからだ。","大切な約束をうっかり忘れてしまった。","うっかりして降りる駅を乗り過ごした。",
        "家の前をうろうろしている人がいる。","クマがうろうろと動き回る。","あっちへうろうろ、こっちへうろうろした。",
        "宿題が多くてうんざりする。","店長の小言にはもううんざりだ。","乗客はうんざりした顔をしている。",
        "地震がきて、戸ががたがた揺れた。","古い机なのでがたがたする。","寒くてがたがた震える。",
        "料理がまずくてがっかりした。", "不合格の知らせにがっかりとさせられた", "仕事がうまくいかなくてがっかりだ", "生徒たちががやがやさわいでいる",
        "パーティ会場はがやがやしていた", "がやがやした店は好きじゃない", "のどがかわいて、からからだ",
        "洗濯物がからからに乾いた", "からから天気の日が続く", "がらがらとシャッターを開ける",
        "地震でへいががらがらと崩れた", "歌いすぎてのどががらがらになった", "家に帰ったらすぐ，がらがらとうがいをする",
        "店員ががらがら声で客を呼びこんでいる", "がんがん工事をする音が聞こえる", "野球のコーチががんがん怒鳴っている",
        "二日酔いで頭ががんがんする", "クーラーをがんがんにきかせる", "毎日朝ごはんをきちんと食べている",
        "本棚に本をきちんと並べる", "家賃を毎月きちんと払う", "財布にお札がぎっしりつまっている",
        "今週は予定がぎっしりだ", "小さな字でぎっしりと書いてある", "家具のサイズをきっちり測る",
        "本棚に本がきっちり入れてある", "借りたお金はきっちり返す", "セールスの電話はきっぱり断る",
        "今日からたばこはきっぱりやめる", "きっぱりとした態度をとる", "空の星がきらきら光る",
        "子供は目がきらきらしている", "パーティにきらきらの服を着ていく", "走っていったらぎりぎり間に合った",
        "合格点ぎりぎりでパスした", "ぎりぎりのお金しか持っていかない", "締め切りぎりぎりにならないと書き始めない",
        "マンガを読んでくすくすと笑っている", "くすくす笑いをしている", "ぐずぐずしていると間に合わない",
        "花粉症で鼻がぐずぐずする", "きのうの夜は、ぐっすり眠れた", "子供がぐっすりと寝ている",
        "最近どうもぐっすり寝られない", "重いドアをぐっと押す", "泣きたくてもぐっとがまんする",
        "冷たいビールをぐっと飲む", "コマがくるくる回る", "カレンダーをくるくる丸めた",
        "くるくるに巻いた髪", "道に迷ってぐるぐる歩き回った", "乗り物がぐるぐる回転する",
        "酔っ払って目がぐるぐる回る", "ロープでぐるぐる巻きにする", "病気でげっそりとやせた",
        "げっそりした表情で帰ってきた", "毎日同じものを食べてげっそりする", "漫画を読んでげらげら笑っている",
        "げらげら笑いすぎてお腹が痛くなった", "人の失敗をげらげらと笑うのは失礼だ", "引き出しの中がごちゃごちゃだ",
        "小さな店がごちゃごちゃと建っている", "ごちゃごちゃした模様は好きじゃない", "映画のストーリーがごちゃごちゃしてわかりにくい",
        "テーブルの上の料理をこっそり食べた", "夜遅くこっそりと家を出た", "ピンポン玉がころころところがる",
        "ころころと太った子犬", "言うことがころころ変わる", "試合にころころ負ける",
        "雷がごろごろなる", "猫がごろごろとのどを鳴らす", "お腹がごろごろする",
        "荷物がごろごろ転がる", "雨がざあざあ降っている", "外はざあざあ降りの大雨だ",
        "一人でさっさと帰ってしまった", "宿題をさっさと片付ける", "５時になったらさっと帰る",
        "テーブルの上をさっと片付ける", "雨がさっと降ってすぐやんだ", "資料にざっと目を通す",
        "バケツの水をざっとかける", "袋に入った米がざっとこぼれた", "ざっと計算して100万円はかかる",
        "長い髪をさっぱりと短くした", "デザートはさっぱり(と)した果物が食べたい", "難しくてさっぱりわからない",
        "髪がさらさらできれいだ", "さらさらの粉雪が降りつもっている", "さらさらっとサインする",
        "砂がさらさら(と)こぼれる", "砂がざらざらと落ちた", "風が強い日は，床がざらざらになる",
        "おばあちゃんの手はざらざらだ", "家の土台がしっかりしている", "子どもが母親にしっかりつかまっている",
        "若いときからしっかり貯金している", "しっかりした計画を立てる", "将来のことをじっくり考えて決める",
        "じっくり煮込んだシチューを作る", "痛くてもじっとがまんする", "心配でじっとしていられない",
        "人をじろじろ見るのは失礼だ", "店員にじろじろ見られた", "薬を飲んですっかり良くなった",
        "宿題のことをすっかり忘れていた", "もうすっかり一人前の大人だ", "部屋を片付けてすっきりした", "今朝は、すっきり起きられた",
        "すっきりしたデザインの洋服", "冷たいものを飲むとすっとする", "音もなくすっと部屋に入ってきた",
        "悪者がつかまって，胸がすっとした", "このペンはすらすら書ける", "難しい問題をすらすら解いた",
        "駐車場に観光バスがずらりと並んでいる", "世界各国のワインをずらりとそろえている", "重い荷物をずるずるとひきずる",
        "ずるずると返事をのばしている", "悪い仲間とずるずる付き合う", "親子で声がそっくりだ",
        "これとそっくりのかばんを持っている", "本物とそっくりに作ってある", "どろぼうが金庫の中身をそっくり盗んだ",
        "ワイングラスをそっと持つ", "うしろの出口からそっと帰る", "この問題には触れずにそっとしておこう",
        "もう遅いからそろそろ帰ろう", "息子もそろそろ結婚を考える歳になった", "足が痛いのでそろそろと歩いている",
        "観光客がぞろぞろ降りてきた", "なべにお湯をたっぷり入れる", "時間がたっぷりある",
        "たっぷりした服を着る", "だぶだぶのズボンが流行する", "お腹の肉がだぶだぶしている",
        "食事の前にちゃんと手を洗う", "小さな子供たちがちゃんと並んでいる", "床がつるつるしていて危ない",
        "つるつるの肌になる", "スピーチをするとき、どきどきした", "走ったあとは、心臓がどきどきする",
        "テストの点を見るときはいつもどきどきだ", "1日中歩き回ってどっと疲れた", "電車から人がどっと降りてくる",
        "ドアをどんどんとたたく音がする", "祭りの太鼓をどんどんたたく", "日本語がどんどん上手になる",
        "駅前にマンションがどんどん建つ", "ジャングルの中をどんどん進む", "赤ちゃんがにこにこ笑っている",
        "ちえ子さんはいつもにこにこしている", "ボーナスをもらってにこにこ顔だ", "携帯メールを見ながらにやにやしている",
        "授業中にやにや笑っていて先生に怒られた", "急ににやにやした顔になった", "プールの底がぬるぬるしている",
        "油がついて，手がぬるぬるすべる", "ぬるぬるした海草が気持ち悪い", "納豆のねばねばが口につく",
        "ねばねばした食べ物は体にいい", "行列がのろのろと進んだ", "のろのろしていると遅れるよ",
        "連休で高速道路はどこものろのろ運転だ", "温泉に入ってのんびりする", "休みの日は、のんびりと過ごしたい",
        "赤ちゃんが手足をばたばたさせる", "暑さで人がばたばた倒れる", "朝はみんなばたばたと出かけていく",
        "忙しくてばたばたしている", "寝不足で頭がはっきりしない", "はっきりしない天気が続く",
        "20年前の友人とばったり会った", "選手はゴールに入ったとたんばったり（と）倒れた", "悪い夢を見てはっと目がさめた",
        "急ブレーキの音がしてはっとした", "車内アナウンスにはっとして飛び降りた", "ぱっと見て決めた",
        "うわさがぱっと広まる", "ボーナスをぱっと使ってしまった", "売り上げがぱっとしない",
        "木の葉がはらはらと散る", "涙がはらはらと落ちた", "人がばらばらと飛び出してきた",
        "家族の食事の時間がばらばらだ", "ジグソーパズルをばらばらにする", "ばらばら死体が発見された",
        "くつをぴかぴかにみがく", "ダイアモンドの指輪がぴかぴか(と)光る", "急に大きな音がしてびっくりした",
        "あの人がどろぼうだったとは、びっくりだ", "おじいさんから、びっくり箱をもらった", "くつのサイズがぴったり合った",
        "子供が母親にぴったりくっついている", "体にぴったりした服を着る", "パンがふっくらと焼けた",
        "モデルにしてはふっくらした体型だ", "ふと後ろを見ると，先生が立っていた", "ふとしたことから知り合った",
        "忘れていた用事をふと思い出した", "寝不足でふらふらする", "酔っ払いがふらふら歩いている",
        "ビデオで撮った画面がふらふら揺れる", "悪い友達にふらふらとついていく", "ふらふらと遊び歩いている",
        "電気のひもがぶらぶら揺れる", "小さな子供が足をぶらぶらさせている", "街をぶらぶらと歩く",
        "こわくて足がぶるぶるする", "犬がぶるぶるっと体をふるわせた", "お腹がすいてぺこぺこだ",
        "電話をかけながら、ぺこぺこ頭を下げる", "社長の前ではぺこぺこする", "アルミの皿がぺこぺこする",
        "外国語がぺらぺらだ", "ぺらぺらよくしゃべる", "ページをぺらぺらとめくる",
        "ぺらぺらの紙", "頭がぼうっとしてきた", "ぼうっとテレビを見ていた",
        "試験に合格してほっとした", "手術が無事に終わってほっと安心した", "頂上に着いてほっと一息つく",
        "ぼろぼろの辞書を使っている", "試合でぼろぼろに負けた", "車がぼろぼろだ",
        "彼はぼんやりした人だ", "遠くの山がぼんやりと見える", "昔のことをぼんやりと覚えている",
        "船酔いで胸がむかむかする", "食べ過ぎて胃がむかむかする", "弟に負けてむかむかした",
        "Ｂ君の作文はめちゃくちゃだ", "地震で家がめちゃくちゃにこわれた", "あの店はめちゃくちゃな値段をつけている",
        "Ｔ大学に合格してめちゃくちゃうれしい", "男の子たちがもりもり食べている", "やる気がもりもりわいてきた",
        "ボディビルの選手は筋肉もりもりだ", "もっとゆっくり話してください", "川の水がゆっくり流れている",
        "みんなゆっくりしたペースで走っている", "おじいさんがよろよろ歩いている", "あっちへよろよろ，こっちへよろよろする",
        "ボクサーがよろよろと倒れる", "初めて海外旅行に行くのでわくわくする", "入学の日をわくわくしながら待っている",
        "『ハリー・ポッター』はわくわくどきどきの映画だ", 
        ]